In [1]:
%load_ext jupyter_black

## [Enums](https://docs.python.org/3/library/enum.html)

> An enumeration is a set of symbolic names (members) bound to unique, constant values. Within an enumeration, the members can be compared by identity, and the enumeration itself can be iterated over.


**Note** Case of Enum Members

Because Enums are used to represent constants we recommend using UPPER_CASE names for enum members, and will be using that style in our examples.


``` python
Python 3.10.4 (main, Jun 29 2022, 12:14:53) [GCC 11.2.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> from enum import Enum
>>> class Color(Enum):
...     RED = 1
... 
>>> Color
<enum 'Color'>
>>> Color.RED
<Color.RED: 1>
>>> Color(1)
<Color.RED: 
```


In [1]:
from enum import IntEnum, Flag, EnumMeta, auto

from typing import TypeVar, Generic, Iterator, NewType

import pandas as pd
import numpy as np


class WXCodeMeta(EnumMeta):
    def __iter__(self: type[IntEnum]) -> Iterator[tuple[str, int]]:
        for member in super().__iter__():
            yield member.name, member.value

    def names(self: type[IntEnum]):
        return tuple(member.name for member in super().__iter__())

    def values(self: type[IntEnum]):
        return tuple(member.value for member in super().__iter__())


class WXCodes(IntEnum, metaclass=WXCodeMeta):
    RA = 1
    TS = 2
    BR = 3
    SN = 4
    FG = 5


df = pd.DataFrame(
    [
        {"WINDSPEED": 30, "WX": "RA"},
        {"WINDSPEED": 15, "WX": "FG"},
        {"WINDSPEED": 35, "WX": pd.NA},
        {"WINDSPEED": 40, "WX": "SN"},
    ]
)


print(
    f"""
{df}

{df.replace(WXCodes.names(), WXCodes.values())}

{df.replace(dict(WXCodes))}
"""
)


   WINDSPEED    WX
0         30    RA
1         15    FG
2         35  <NA>
3         40    SN

   WINDSPEED    WX
0         30     1
1         15     5
2         35  <NA>
3         40     4

   WINDSPEED    WX
0         30     1
1         15     5
2         35  <NA>
3         40     4



In [3]:
print(
    f"""\
tuple:
{tuple(WXCodes)}

list:
{list(WXCodes)}

dict:
{dict(WXCodes)}

numpy array:
{np.array(WXCodes, dtype=object)}

pandas dataframe:
{pd.DataFrame(tuple(WXCodes), columns=["names", "values"])}"""
)

tuple:
(('RA', 1), ('TS', 2), ('BR', 3), ('SN', 4), ('FG', 5))

list:
[('RA', 1), ('TS', 2), ('BR', 3), ('SN', 4), ('FG', 5)]

dict:
{'RA': 1, 'TS': 2, 'BR': 3, 'SN': 4, 'FG': 5}

numpy array:
[['RA' 1]
 ['TS' 2]
 ['BR' 3]
 ['SN' 4]
 ['FG' 5]]

pandas dataframe:
  names  values
0    RA       1
1    TS       2
2    BR       3
3    SN       4
4    FG       5


In [34]:
import urllib.parse
from collections import ChainMap
from enum import Enum, EnumMeta, auto
from typing import Iterator, TypeVar, Iterable, Generator, NewType


Names = TypeVar("Names", str, list[str], tuple[str])
StrGenerator = NewType("Generator[str, ...]", Generator[str, None, None])


class StrEnum(str, Enum):
    name: str
    value: str

    def __iter__(self: type[Enum]) -> StrGenerator:
        yield from super().__iter__()


def _urlencode(items: Iterable["QueryEnum"]):
    return urllib.parse.urlencode(
        tuple((member.__query_name__, member.value) for member in items)
    )


class QueryMap(ChainMap):
    def __str__(self) -> str:
        return _urlencode(super().__iter__())

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({', '.join(member.__repr__() for member in super().__iter__())})"

    def __and__(self, __o: "QueryMap"):
        return str(QueryMap(self, __o))


class QueryEnumMeta(EnumMeta):
    def __values__(self: type[StrEnum]) -> StrGenerator:
        yield from (member.value for member in super().__iter__())

    def __names__(self: type[StrEnum]) -> StrGenerator:
        yield from (member.name for member in super().__iter__())

    def __string_contains__(self, __o: str) -> bool:
        return __o in self.__values__() or __o in self.__names__()

    def __str__(self) -> str:
        return _urlencode(super().__iter__())

    def __getitem__(self, names: Names) -> QueryMap:
        if not isinstance(names, list):
            names = (names,)
        return QueryMap(tuple(self._member_map_[name] for name in names))

    def __contains__(self, __o: object) -> bool:
        if isinstance(__o, str):
            return self.__string_contains__(__o)
        return super().__contains__(__o)

    def __eq__(self, __o: object) -> bool:
        if isinstance(__o, str):
            return self.__string_contains__(__o)
        return super().__eq__(__o)

    def __hash__(self) -> int:
        return super().__hash__()

    def __and__(self, __o: StrEnum):
        return QueryMap(self, __o)

    def names(self):
        return tuple(self.__names__())

    def values(self):
        return tuple(self.__values__())


class QueryEnum(StrEnum, metaclass=QueryEnumMeta):
    def _generate_next_value_(name: str, *_) -> str:
        return name

    def __repr__(self):
        return f"{self.__query_name__}={self.value}"

    @classmethod
    def __lt__(self, __o: "QueryEnum") -> bool:
        return self.__query_name__ < __o.__query_name__

    @classmethod
    @property
    def __query_name__(cls) -> str:
        return cls.__name__.lower()


class Models(QueryEnum):
    GALWEM = auto()
    NAM = auto()
    GFS = auto()
    WRF_17K = "WRF-1.7k"


class Parameters(QueryEnum):
    wind_direction = auto()
    wind_speed = auto()
    wind_gust = auto()
    visibility = auto()
    present_wx = auto()
    ten_meter_temp = "10_m_temp"

localhost = "http://localhost:8080"

def main():

    # inherited methods
    assert Parameters("10_m_temp") == Parameters.ten_meter_temp
    assert Parameters.ten_meter_temp == "10_m_temp"
    assert Parameters.ten_meter_temp.upper() == "10_M_TEMP"
    # string methods
    assert (
        f"{localhost}?{QueryMap(Parameters,Models)}"
        == f"{localhost}?{Parameters & Models}"
        == f"{localhost}?{Models}&{Parameters}" 
    )
    # in
    assert "GALWEM" in Models
    assert Models.GALWEM in Models
    assert Models in ("GALWEM", "NAM")
    
    if not "BAD_USER_REQUEST" in Models:
        print("THAT MODEL IS NOT AVALIABLE")


    assert (
        f"{localhost}?{Models & Parameters}"
        == f"{localhost}?{QueryMap(Models,Parameters)}"
    )
    print(f"""
{localhost}?{Models['GALWEM'] & Parameters}

{localhost}?{Models[['GALWEM', 'NAM']] & Parameters}

{localhost}?{Models[['GALWEM', 'NAM']] & Parameters['wind_direction']}
    """)


if __name__ == "__main__":
    main()


THAT MODEL IS NOT AVALIABLE

http://localhost:8080?parameters=wind_direction&parameters=wind_speed&parameters=wind_gust&parameters=visibility&parameters=present_wx&parameters=10_m_temp&models=GALWEM

http://localhost:8080?parameters=wind_direction&parameters=wind_speed&parameters=wind_gust&parameters=visibility&parameters=present_wx&parameters=10_m_temp&models=GALWEM&models=NAM

http://localhost:8080?parameters=wind_direction&models=GALWEM&models=NAM
    


In [27]:
import numpy as np

a = np.array(Models, dtype=object)
mask = (a == "GALWEM") | (a == "NAM")
f"{localhost}?{QueryMap(a[mask])}"

'http://localhost:8080?models=GALWEM&models=NAM'

In [28]:
import pandas as pd


s = pd.Series(tuple(Parameters), index=Parameters.names())
f"{localhost}?{QueryMap(s[s.str.contains('wind')])}"

'http://localhost:8080?parameters=wind_direction&parameters=wind_speed&parameters=wind_gust'

In [29]:
def f(params: Parameters):
    return Parameters[params]


f(["wind_direction", "wind_speed"])

QueryMap(parameters=wind_direction, parameters=wind_speed)